In [26]:
import numpy as np
import pandas as pd
import edhec_risk_kit_206 as erk
%load_ext autoreload
%autoreload 2

inds = ['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']
ind_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["2013":"2018"][inds]
ind_mcap = erk.get_ind_market_caps(49, weights=True)["2013":"2018"][inds]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
ind_mcap.sum()

Hlth     0.396624
Fin      1.751218
Whlsl    0.937026
Rtail    5.057213
Food     1.241093
dtype: float64

In [45]:
cw=erk.weight_cw(ind_rets,ind_mcap)
cw

Hlth     0.041663
Fin      0.175362
Whlsl    0.097411
Rtail    0.546388
Food     0.139176
Name: 2013-01, dtype: float64

In [43]:
rho =ind_rets.corr()
rho

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,1.000000,0.524886,0.650900,0.574227,0.371698
Fin,0.524886,1.000000,0.777653,0.595332,0.343016
Whlsl,0.650900,0.777653,1.000000,0.743754,0.541244
Rtail,0.574227,0.595332,0.743754,1.000000,0.575063
Food,0.371698,0.343016,0.541244,0.575063,1.000000


In [44]:
vols = ind_rets.std()*np.sqrt(12)
vols

Hlth     0.162801
Fin      0.167753
Whlsl    0.135644
Rtail    0.141238
Food     0.125401
dtype: float64

In [46]:
sigma_prior = vols.dot(vols.T) * rho
sigma_prior

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,0.108718,0.057064,0.070764,0.062429,0.040410
Fin,0.057064,0.108718,0.084545,0.064723,0.037292
Whlsl,0.070764,0.084545,0.108718,0.080859,0.058843
Rtail,0.062429,0.064723,0.080859,0.108718,0.062519
Food,0.040410,0.037292,0.058843,0.062519,0.108718


In [47]:
pi = erk.implied_returns(delta=2.5, sigma=sigma_prior, w=cw)
pi

Hlth     0.152910
Fin      0.175580
Whlsl    0.201836
Rtail    0.224827
Food     0.158115
Name: Implied Returns, dtype: float64

In [42]:
pi[pi==pi.max()]

Rtail    0.224827
Name: Implied Returns, dtype: float64

In [21]:
pi[pi==pi.min()]

Hlth    0.15291
Name: Implied Returns, dtype: float64

In [30]:
# Hlth will outperform other Rtail and Whlsl by 3%
q = pd.Series([.03]) # just one view
# start with a single view, all zeros and overwrite the specific view
p = pd.DataFrame([0.]*len(['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']),
                 index=['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']).T

w_re =  cw.loc["Rtail"]/(cw.loc["Rtail"]+cw.loc["Whlsl"])
w_wh =  cw.loc["Whlsl"]/(cw.loc["Rtail"]+cw.loc["Whlsl"])
p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Whlsl'] = -w_wh
p.iloc[0]['Rtail'] = -w_re
p

,Hlth,Fin,Whlsl,Rtail,Food
0,1.0,0.0,-0.151306,-0.848694,0.0


In [31]:
# Hlth will outperform other Rtail and Whlsl by 5%
q = pd.Series([.05]) # just one view
# start with a single view, all zeros and overwrite the specific view
p = pd.DataFrame([0.]*len(['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']), index=['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']).T

w_re =  cw.loc["Rtail"]/(cw.loc["Rtail"]+cw.loc["Whlsl"])
w_wh =  cw.loc["Whlsl"]/(cw.loc["Rtail"]+cw.loc["Whlsl"])
p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Whlsl'] = -w_wh
p.iloc[0]['Rtail'] = -w_re
p

,Hlth,Fin,Whlsl,Rtail,Food
0,1.0,0.0,-0.151306,-0.848694,0.0


In [49]:
delta = 2.5
tau = 0.03 # from Footnote 8
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(cw, sigma_prior, p, q, tau = tau)
(bl_mu*100).round(1)

Hlth     18.5
Fin      16.8
Whlsl    19.2
Rtail    19.5
Food     14.3
dtype: float64

In [50]:
delta = 2.5
tau = 0.05 # from Footnote 8
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(cw, sigma_prior, p, q, tau = tau)
(bl_mu*100).round(1)

Hlth     18.5
Fin      16.8
Whlsl    19.2
Rtail    19.5
Food     14.3
dtype: float64

In [36]:
w_eq  = erk.w_msr(delta*sigma_prior, pi, scale=False)
w_eq

Hlth     0.041663
Fin      0.175362
Whlsl    0.097411
Rtail    0.546388
Food     0.139176
dtype: float64